In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from estnltk import Span, Layer, Text

text = Text('Tere, \n maailm')
layer_0 = Layer('layer_0', attributes=['attr_1', 'attr_2'], ambiguous=True)
layer_0.add_span(Span(start=0,  end=1,  legal_attributes=['attr_1', 'attr_2'], attr_1='A1', attr_2='11'))
layer_0.add_span(Span(start=0,  end=1,  legal_attributes=['attr_1', 'attr_2'], attr_1='A2', attr_2='22'))

text['layer_0'] = layer_0
text.layer_0.display()

T ere, maailm

In [3]:
text.layer_0.display()

T ere, maailm

In [4]:
Text('Aias sadas saia').tag_layer(['morph_analysis'])

Text(text="Aias sadas saia")

In [5]:
from estnltk.tests import new_text

new_text(5).layer_1.display()

Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd tuhat.

In [6]:
t = Text ('''Täna ennelõunal oli relvast lasu teinud mehe korteri all elav naaber oma igapäevaste toimetuste juures.
Ta rääkis, et käis eile õhtul korraks poest piima ostmas ning jättis lapse kontrolltööks õppima, kui vahepeal oli käinud
pauk otse tüdruku pea kohal. "Ei saa kaheksa minutit poeski ära käia, kohe juhtub midagi," seletas naaber.''')

t.tag_layer(["morph_analysis"])

t.morph_analysis[0]

t.morph_analysis.display()

Täna ennelõunal oli relvast lasu teinud mehe korteri all elav naaber oma igapäevaste toimetuste juures . Ta rääkis , et käis eile õhtul korraks poest piima ostmas ning jättis lapse kontrolltööks õppima , kui vahepeal oli käinud pauk otse tüdruku pea kohal . " Ei saa kaheksa minutit poeski ära käia , kohe juhtub midagi , " seletas naaber .